# T1090.004 Domain Fronting

-----------------------------------------------------------------------

## Technique Description

Adversaries may take advantage of routing schemes in Content Delivery Networks (CDNs) and other services which host multiple domains to obfuscate the intended destination of HTTPS traffic or traffic tunneled through HTTPS. (Citation: Fifield Blocking Resistent Communication through domain fronting 2015) Domain fronting involves using different domain names in the SNI field of the TLS header and the Host field of the HTTP header. If both domains are served from the same CDN, then the CDN may route to the address specified in the HTTP header after unwrapping the TLS header. A variation of the the technique, "domainless" fronting, utilizes a SNI field that is left blank; this may allow the fronting to work even when the CDN attempts to validate that the SNI and HTTP Host fields match (if the blank SNI fields are ignored).

For example, if domain-x and domain-y are customers of the same CDN, it is possible to place domain-x in the TLS header and domain-y in the HTTP header. Traffic will appear to be going to domain-x, however the CDN may route it to domain-y.

## Technique Detection

If SSL inspection is in place or the traffic is not encrypted, the Host field of the HTTP header can be checked if it matches the HTTPS SNI or against a blocklist or allowlist of domain names. (Citation: Fifield Blocking Resistent Communication through domain fronting 2015)

-----------------------------------------------------------------------

### Tactics:

  *   Command-And-Control

### Platforms:

  * Linux

  * macOS

  * Windows

### Data Sources:

  * **Network Traffic:** Network Traffic Content

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
| APT29 | [APT29](https://attack.mitre.org/groups/G0016) has used the meek domain fronting plugin for [Tor](https://attack.mitre.org/software/S0183) to hide the destination of C2 traffic.(Citation: Mandiant No Easy Breach)| 
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1090/004)

  * [Capec](https://capec.mitre.org/data/definitions/481.html)

  * [Fifield Blocking Resistent Communication Through Domain Fronting 2015](http://www.icir.org/vern/papers/meek-PETS-2015.pdf), David Fifield, Chang Lan, Rod Hynes, Percy Wegmann, and Vern Paxson. (2015). Blocking-resistant communication through domain fronting. Retrieved November 20, 2017.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 06 July 2022

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres, CTR Emily Porras

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Adversaries may use various open-source tools such as meek and TOR to obfuscate the destination of C2 traffic.

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| APT29 |  | 1 |

## Detection Blindspots

- Encrypted traffic may make it difficult to identify details of the connection. Keeping the true destination of the traffic out of view from the sensor. Use meta data surrounding the connections and TLS information if possible.
- Sensor placement may also play a role in not being able to identify this specific TTP.

## Analytical References

  * [No Easy Breach](https://www.slideshare.net/MatthewDunwoody1/no-easy-breach-derby-con-2016)
  * [Packetlabs - Introduction to Domain Fronting](https://www.packetlabs.net/introduction-to-domain-fronting/)
  * [Meek (github)](https://github.com/arlolra/meek)
  * [Meek 2015 (icir)](http://www.icir.org/vern/papers/meek-PETS-2015.pdf)
  * [How to use meek plugin (torproject)](https://blog.torproject.org/how-use-meek-pluggable-transport)
  * [APT29 Domain Fronting 2017 (fireeye)](https://www.fireeye.com/blog/threat-research/2017/03/apt29_domain_frontin.html)



-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes
- Refer to T1090.003 Multip-Hop Proxy



-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

- If SSL/TLS inspection is not allowed this may be difficult to detect and attention should be placed elsewhere around the connections metadata. 
- If SSL inspection is in place or the traffic is not encrypted, the Host field of the HTTP header can be checked if it matches the HTTPS SNI or against a blocklist or allowlist of domain names.
- The key idea of domain fronting is the use of different domain names at different layers of communication. In an HTTPS request, the destination domain name appears in three relevant places: in the DNS query, in the TLS Server Name Indication (SNI) extension, and in the HTTP Host header. Ordinarily, the same domain name appears in all three places.
- In a domain-fronted request, however, the DNS query and SNI carry one name (the “front domain”), while the HTTP Host header, hidden from the censor by HTTPS encryption, carries another (the covert, forbidden destination).
- Once the operator has keyed in on a domain of interest, analysis of every hop is crucial. This hunt can be time consuming and difficult if the network enviroment is misconfigured or unconventional (i.e., VDI)
- Adversaries will utilize CDNs to hide communications to C2 servers as well as hide the transfer of data in/out of the victim network

![T1090004](../../Images/T1090.004_Domain_Fronting.png)
#### Analytic 1

  * **Information:** Identify traffic riding content delievery networks in order to hide the true intended destination.

  * **Source:** PCAP, sessions*

  * **Tool:** Arkime, Kibana

  * **Notes:** This TTP can be challenging to detect if the CDN is extensive and highly annonymous (i.e., TOR) examine TLS and HTTPS headers for references to different domains. 

  * **Query Arkime:** 
       - `host.dns == [Domain of interest]`

       - `host.http == [HTTP host of interest]`

       - `protocols == tls`
           * ssl protocol is another vaiable option for adversaries to utilize for encrypted traffic

       - `http.uri == [URI of interest]`
       
  * **Query Kibana:** 
       - `dns.host : [domain of interest]`

       - `http.host : [HTTP host of interest]`

       - `protocol : tls`

       - `http.uri : [URI of interest]`
